# Design Patterns in CZ2006

In [6]:
from abc import ABC, abstractmethod
import random

## Strategy Pattern

The Strategy Pattern defines a family of algorithms, encapsulates each one, and makes them interchangeable. Strategy lets the algorithm vary independently from the clients that use it.

**It enables selecting an algorithm at runtime.**

![](uml-diagrams/strategy-pattern.png)

Focus on the has-a relationship between Context and AbstractStrategy. It's kind of like letting a class have a behavior as its attribute. E.g. Context could have a ContextStrategyA algorithm as its attribute. When it wants to do carry out its business logic utilizing the algorithm, it simply calls performStrategy. It doesn't know how to perform the algorithm - that's for AbstractStrategy to take care of.

### Benefits of Strategy Pattern

???

### Skelly Code

In [12]:
"""
The class that the client is interested in.
"""
class Context:
    # Pass a Strategy object into the constructor
    def __init__(self, strategy):
        self.strategy = strategy
    
    # Allows the replacement of a Strategy object at runtime
    def setStrategy(self, strategy):
        self.strategy = strategy
    
    # Instead of implementing multiple versions of doSomething on its own,
    # The work is delegated to the Strategy object
    def doBusinessLogic(self):
        self.strategy.doSomething()

"""
Interface declaring operations common to all versions of some algorithm.
Context uses this interface to call the algorithm defined by Concrete Strategies.
Context doesn't know how Strategy will do it, but it knows it'll get done.
"""    
class AbstractStrategy(ABC):
    @abstractmethod
    def doSomething(self):
        pass

"""
Implementations of AbstractStrategy.
"""
class ConcreteStrategyA(AbstractStrategy):
    def doSomething(self):
        print("I'm ConcreteStrategyA!")
        
class ConcreteStrategyB(AbstractStrategy):
    def doSomething(self):
        print("I'm ConcreteStrategyB!")
        
c = Context(ConcreteStrategyA())
c.doBusinessLogic()

print("---")

c.setStrategy(ConcreteStrategyB())
c.doBusinessLogic()

I'm ConcreteStrategyA!
---
I'm ConcreteStrategyB!


### Example 1: Duckies

In [8]:
class FlyBehavior(ABC):
    def fly(self):
        pass
    
class FlyWithWings(FlyBehavior):
    def fly(self):
        print("I'm flying!")
        
class FlyNoWay(FlyBehavior):
    def fly(self):
        print("I can't fly. :(");

In [9]:
class QuackBehavior(ABC):
    @abstractmethod
    def quack(self):
        pass
    
class Quack(QuackBehavior):
    def quack(self):
        print("Quack!");
        
class MuteQuack(QuackBehavior):
    def quack(self):
        print("...")
        
class Squeak(QuackBehavior):
    def quack(self):
        print("Squeak~")

In [10]:
class Duck(ABC):
    @abstractmethod
    def display(self):
        pass
    
    def performFly(self):
        self.flyBehavior.fly()
    
    def performQuack(self):
        self.quackBehavior.quack()
        
    def swim(self):
        print("All ducks float, even decoys!")
    
    # Methods to change the behavior of a duck on the fly
    def setFlyBehavior(self, fb):
        self.flyBehavior = fb
        
    def setQuackBehavior(self, qb):
        self.quackBehavior = qb

class MallardDuck(Duck):
    def __init__(self):
        self.quackBehavior = Quack()
        self.flyBehavior = FlyWithWings()
    
    def display(self):
        print("I'm a mallard duck! uwu")

In [11]:
mallard = MallardDuck()
mallard.performQuack()
mallard.performFly()

print("---")

mallard.setFlyBehavior(FlyNoWay())
mallard.performFly()

Quack!
I'm flying!
---
I can't fly. :(


## Observer Pattern

The Observer Pattern defines a one-to-many dependency between objects so that when one object changes state, all of its dependents are notified and updated automatically.

### Pull Model (2-Way Communication)

- When the subject's state changes, it notifies its observers **that its state has changed**
- If the observer is interested, it can call back for details by calling subject.getData()


### Push Model (1-Way Communication)

- When the subject's state changes, it notifies its observers **by sending detailed information about the update**, whether or not the Observer is interested or not
- Hence, the observer never needs to call back



### Benefits of Observer Pattern

- Loose coupling

### Skelly Code

In [36]:
class AbstractSubject(ABC):
    @abstractmethod
    def register(self, observer):
        pass
    
    @abstractmethod
    def unregister(self, observer):
        pass
    
    @abstractmethod
    def notify(self):
        pass
    
class AbstractObserver(ABC):
    @abstractmethod
    def update(self, subject):
        pass

### Pull Model

In [38]:
class ConcreteSubject(AbstractSubject):
    state = None
    observers = []
    
    def register(self, observer):
        self.observers.append(observer)
        
    def unregister(self, observer):
        self.observers.remove(observer)
    
    def notify(self):
        for observer in self.observers:
            observer.update(self) # Subject passed as parameter
            
    def getState(self):
        return self.state
            
    def doBusinessLogic(self):
        self.state = random.randint(0, 10)
        self.notify() # Notify when state changes
        
class ConcreteObserverA(AbstractObserver):
    def update(self, subject):        
        print(f"> ConcreteObserverA was notified of Subject's new state: {subject.getState()}")
    
class ConcreteObserverB(AbstractObserver):
    def update(self, subject):
        print(f"> ConcreteObserverB was notified of Subject's new state: {subject.getState()}")
            
subject = ConcreteSubject()

# Register new observers
observerA = ConcreteObserverA()
subject.register(observerA)
observerB = ConcreteObserverB()
subject.register(observerB)

subject.doBusinessLogic()

print("---")

subject.unregister(observerA) # Unregister Observer A
subject.doBusinessLogic()

> ConcreteObserverA was notified of Subject's new state: 4
> ConcreteObserverB was notified of Subject's new state: 4
---
> ConcreteObserverB was notified of Subject's new state: 8


### Push Model

Focus on the difference in the subject's **notify() method**, and that the observers' **update() methods** take in the subject's state - not the subject object itself.

In [37]:
class ConcreteSubject(AbstractSubject):
    state = None
    observers = []
    
    def register(self, observer):
        self.observers.append(observer)
        
    def unregister(self, observer):
        self.observers.remove(observer)
    
    def notify(self):
        for observer in self.observers:
            observer.update(self.state) # State passed in instead of Subject
            
    def doBusinessLogic(self):
        self.state = random.randint(0, 10)
        self.notify()
        
class ConcreteObserverA(AbstractObserver):
    def update(self, state):        
        print(f"> ConcreteObserverA was notified of Subject's new state: {state}")
    
class ConcreteObserverB(AbstractObserver):
    def update(self, state):
        print(f"> ConcreteObserverB was notified of Subject's new state: {state}")
            
subject = ConcreteSubject()

# Register new observers
observerA = ConcreteObserverA()
subject.register(observerA)
observerB = ConcreteObserverB()
subject.register(observerB)

subject.doBusinessLogic()

print("---")

subject.unregister(observerA) # Unregister Observer A
subject.doBusinessLogic()

> ConcreteObserverA was notified of Subject's new state: 7
> ConcreteObserverB was notified of Subject's new state: 7
---
> ConcreteObserverB was notified of Subject's new state: 7
